In [4]:
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

import PyPDF2
import os
import json
import pandas as pd
import traceback


ModuleNotFoundError: No module named 'google'